In [ ]:
import math
import numpy as np
import math
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import re
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import random
import requests
from wikidata.client import Client
from itertools import islice

In [ ]:
#Save the two dataset to csv
silverdataset = pd.read_csv('silver.csv')
golddataset = pd.read_csv('gold.csv')

In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

def extract_wikipedia_links(entity_id):
    client = Client()
    print(extract_entity_id(entity_id))
    item = client.get(extract_entity_id(entity_id), load=True)
    sitelinks = item.data.get("sitelinks", {})

    # Filter only Wikipedia sitelinks and print them
    #print("Wikipedia pages available for this entity:\n")

    results = []

    for site_key, site_data in sitelinks.items():
        if site_key.endswith("wiki") and not site_key.startswith("commons"):
            lang = site_key.replace("wiki", "")
            title = site_data["title"]
            url = f"https://{lang}.wikipedia.org/wiki/{title.replace(' ', '_')}"
            #print(f"{lang}: {url}")
            results.append([entity_id, lang, url])
    return pd.DataFrame(results, columns=['entity', 'language', 'url'])

# Funzione per salvare progressivamente i risultati in un file CSV
def save_wikipedia_links(file_name):
    output_file_name = "wikipedia_links_"+file_name
    # Controlla se il file di output esiste già
    if os.path.exists(output_file_name):
        # Carica il file esistente
        existing_df = pd.read_csv(output_file_name)
        # Raccoglie le keyword già trattate
        processed_keywords = existing_df['entity'].unique().tolist()
        print(len(processed_keywords), "keywords già trattate.")
    else:
        #Creo un nuovo dataframe
        existing_df = pd.DataFrame(columns=['entity', 'language', 'url'])
        processed_keywords = []
    
    # Carica il dataset di input
    data = pd.read_csv(file_name)
    df = pd.DataFrame(data, columns=["item"]) #item = wikidata link

    # Lista per raccogliere i risultati
    results = []

    # Cicla su ogni entità
    for idx, row in df.iterrows():
        keyword = row["item"]
        
        # Se la keyword è già stata trattata, salta
        if keyword in processed_keywords:
            print(keyword," già trattata, salto.")
            continue
        
        #time.sleep(random.randint(2, 6))
        print("Elaboro: ", keyword)
        #Extract data
        trend_data = extract_wikipedia_links(keyword)
        # Aggiungi i nuovi risultati se non sono vuoti
        if not trend_data.empty:
            results.append(trend_data)
        
        # Scrivi progressivamente i risultati
        if results:
            new_results = pd.concat(results, ignore_index=True)
            new_results.to_csv(output_file_name, mode='a', index=False)
            results = []  # Resetta la lista dei risultati per non aggiungere più volte
        
        # Aggiungi la keyword alla lista delle trattate
        processed_keywords.append(keyword)

    print("Processing completato")


In [ ]:
save_wikipedia_links('silver.csv')